In [1]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords
import os, codecs
%matplotlib inline
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn import metrics as skmetrics

2017-06-17 09:15:56,971 : INFO : 'pattern' package found; tag filters are available for English
/usr/lib64/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Model 1 evaluation

In [2]:
model = gensim.models.Word2Vec.load('/home/bahbbc/workspace/masters-big5/models/tweet50-600.model')
model.init_sims(replace=True)

2017-06-17 09:15:57,606 : INFO : loading Word2Vec object from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-06-17 09:16:03,783 : INFO : loading wv recursively from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.* with mmap=None
2017-06-17 09:16:03,784 : INFO : loading syn0 from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.wv.syn0.npy with mmap=None
2017-06-17 09:16:11,653 : INFO : setting ignored attribute syn0norm to None
2017-06-17 09:16:11,655 : INFO : loading syn1neg from /home/bahbbc/workspace/masters-big5/models/tweet50-600.model.syn1neg.npy with mmap=None
2017-06-17 09:16:47,200 : INFO : setting ignored attribute cum_table to None
2017-06-17 09:16:47,379 : INFO : loaded /home/bahbbc/workspace/masters-big5/models/tweet50-600.model
2017-06-17 09:18:16,509 : INFO : precomputing L2-norms of word weight vectors


In [3]:
print(model.most_similar(positive=['rss', 'haha']))

[(u'hehe', 0.8554530143737793), (u'hahaha', 0.8513627052307129), (u'hahah', 0.8424988389015198), (u'rs', 0.8193700909614563), (u'kk', 0.8075661063194275), (u'rsrs', 0.7939968109130859), (u'hahahah', 0.7900516986846924), (u'kkk', 0.7897874712944031), (u'hehehe', 0.7863951921463013), (u'hahahaha', 0.7844202518463135)]


In [4]:
print(model.most_similar(positive=[':D', ':/']))

[(u':(', 0.795927882194519), (u':p', 0.7728939056396484), (u':s', 0.763850748538971), (u'://', 0.7579135298728943), (u':))', 0.7576685547828674), (u'=/', 0.7541918754577637), (u';/', 0.7447569370269775), (u':S', 0.7422836422920227), (u':P', 0.7380853891372681), (u':\\', 0.7342033386230469)]


In [5]:
print(model.most_similar(positive=['fofo', 'cute']))

[(u'fofinho', 0.7343779802322388), (u'fofo,', 0.6787629127502441), (u'meigo', 0.6691230535507202), (u'lindinho', 0.6611505746841431), (u'kawaii', 0.6287035942077637), (u'fofo!', 0.6275116801261902), (u'fofis', 0.6228358149528503), (u'fofo.', 0.6119152903556824), (u'fofo"', 0.6110981702804565), (u'bonitinho', 0.6054898500442505)]


In [6]:
num_features = model.wv.syn0.shape[1]

In [7]:
df = pd.read_csv('~/personality-normalized-v2-word2vec.csv', encoding='utf-8')
df.shape

(1039, 186)

In [8]:
df.extraversion_m.value_counts()

0    529
1    510
Name: extraversion_m, dtype: int64

In [9]:
train_w2v_data, test_w2v_data = train_test_split(df, test_size=0.3, random_state=42)

In [10]:
def predict(vectorizer, classifier, data):
    data_features = vectorizer.transform(data['formatted_text'])
    predictions = classifier.predict(data_features)
    target = data['extraversion_m']
    evaluate_prediction(predictions, target)

In [11]:
def word_averaging(wv, words):
    all_words, mean = set(), []
    #print words.shape
    for word in words:
        if isinstance(word, np.ndarray):
            mean.append(word)
        elif word in wv.wv.vocab:
            mean.append(wv.wv.syn0norm[wv.wv.vocab[word].index])
            all_words.add(wv.wv.vocab[word].index)
        #print mean

    if not mean:
        logging.warning("cannot compute similarity with no input %s", words)
        # FIXME: remove these examples in pre-processing
        return np.zeros(num_features,)

    mean = gensim.matutils.unitvec(np.array(mean).mean(axis=0)).astype(np.float32)
    return mean

def  word_averaging_list(wv, text_list):
    return np.vstack([word_averaging(wv, review) for review in text_list ])

In [12]:
def w2v_tokenize_text(text):
    tokens = []
    if text is np.nan:
        return []
    for sent in nltk.sent_tokenize(text, language='portuguese'):
        for word in nltk.word_tokenize(sent, language='portuguese'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [13]:
test_tokenized = test_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values
train_tokenized = train_w2v_data.apply(lambda r: w2v_tokenize_text(r['formatted_text']), axis=1).values

In [14]:
%%time
X_train_word_average = word_averaging_list(model,train_tokenized)
X_test_word_average = word_averaging_list(model,test_tokenized)

2017-06-17 09:20:03,706 : WARNING : cannot compute similarity with no input [u'pokersport']
2017-06-17 09:20:04,595 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:05,291 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:05,301 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:06,432 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:06,799 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:07,353 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:08,368 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:08,491 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:08,805 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:09,025 : WARNING : cannot compute similarity with no input []
2017-06-17 09:20:09,102 : WARNING : cannot compute similarity with no input []


CPU times: user 5.44 s, sys: 347 ms, total: 5.79 s
Wall time: 8.48 s


In [15]:
del model

In [16]:
X_train_word_average.shape

(727, 600)

In [17]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib

force = False
model_trainer = RandomizedSearchCV(
    n_iter=1, 
    estimator=RandomForestClassifier(),
    param_distributions={
        "criterion": ["gini"],
        "n_estimators": [1000],
        "max_features": ["log2"],
        "max_depth": [None],
        "bootstrap": [True],
        "oob_score": [True],
        "class_weight": ["balanced"],
        "random_state": [42]
    },
    scoring="f1",
    verbose=True,
    refit=True,
    cv=10,
    n_jobs=-1
)

In [18]:
%%time
model_trainer.fit(X_train_word_average, train_w2v_data['extraversion_m'])
model = model_trainer.best_estimator_

Fitting 10 folds for each of 1 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   25.0s finished


CPU times: user 5.27 s, sys: 380 ms, total: 5.65 s
Wall time: 32.5 s


In [19]:
joblib.dump(model, '/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl')

['/home/bahbbc/workspace/masters-big5/extraversion-best-random-tree-model1.pkl']

In [20]:
yp = model.predict(X_test_word_average)
yt = test_w2v_data['extraversion_m']

In [21]:
pd.DataFrame(
    index=pd.Index([0, 1], name="y_true"),
    columns=pd.Index([0, 1], name="y_pred"),
    data=skmetrics.confusion_matrix(y_true=yt, y_pred=yp)
)

y_pred,0,1
y_true,,
0,101,41
1,86,84


In [22]:
print skmetrics.classification_report(y_true=yt, y_pred=yp)

             precision    recall  f1-score   support

          0       0.54      0.71      0.61       142
          1       0.67      0.49      0.57       170

avg / total       0.61      0.59      0.59       312



In [23]:
accuracy_score(yt, yp)

0.59294871794871795

In [24]:
print '--------- TRAIN -----------------'

--------- TRAIN -----------------


In [25]:
ytp = model.predict(X_train_word_average)
ytt = train_w2v_data['extraversion_m']

In [26]:
print skmetrics.classification_report(y_true=ytt, y_pred=ytp)

             precision    recall  f1-score   support

          0       1.00      1.00      1.00       387
          1       1.00      1.00      1.00       340

avg / total       1.00      1.00      1.00       727



In [27]:
print accuracy_score(ytt, ytp)

0.998624484182
